<a href="https://colab.research.google.com/github/jykim9280/KISTI_AI/blob/minseo/%EA%B3%B5%EB%AA%A8%EC%A0%84_%EC%BD%94%EB%9E%A9ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install transformers datasets
# !pip install accelerate  # PyTorch 사용을 위한 가속화
# !pip install transformers datasets
# !pip install torch accelerate


In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import os
import sys
import urllib.request
import pandas as pd
import requests
import json
import glob

import warnings
# 경고 메시지 무시 설정
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


In [5]:
current_directory = os.getcwd()
print(f"현재 작업 디렉토리: {current_directory}")

현재 작업 디렉토리: /content


In [5]:
from google.colab import files
uploaded = files.upload()  # 파일 업로드


Saving 라벨링데이터2.json to 라벨링데이터2.json


In [ ]:
from datasets import load_dataset

In [7]:
import json
import os
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# 1. GPT-2 모델과 토크나이저 로드
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 2. JSON 파일 경로 (업로드 대신 직접 경로에서 로드)
json_file_path = "/content/라벨링데이터2.json"  # Colab 환경에 있는 파일 경로

# 3. JSON 파일을 GPT-2 학습용 텍스트 데이터로 변환
def convert_json_to_gpt2_format(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    qa_pairs = []
    for item in data:
        question = item.get("question")
        answer = item.get("answer")
        if question and answer:
            qa_pairs.append(f"질문: {question}\n답변: {answer}")

    train_data = "\n\n".join(qa_pairs)
    train_file_path = "/content/gpt2_qa_train_data.txt"

    with open(train_file_path, 'w', encoding='utf-8') as f:
        f.write(train_data)

    return train_file_path

# JSON 파일을 텍스트 형식으로 변환하여 저장
train_file_path = convert_json_to_gpt2_format(json_file_path)

# 4. 텍스트 데이터셋을 로드
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

train_dataset = load_dataset(train_file_path, tokenizer)

# 5. 데이터 컬레이터 설정 (언어 모델 학습용)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT는 마스크드 언어 모델이 아니므로 False로 설정
)

# 6. 학습 하이퍼파라미터 설정 (시간을 줄이기 위해 에폭 수와 배치 크기를 조정)
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",  # 모델 저장 디렉토리
    overwrite_output_dir=True,
    num_train_epochs=3,             # 학습할 에폭 수 (시간을 줄이기 위해 1로 설정) # 나중에는 3이상으로 바꾸기
    per_device_train_batch_size=32,  # 배치 크기를 늘려 시간 단축
    save_steps=5_000,               # 모델을 저장할 단계 간격
    save_total_limit=2,             # 저장할 체크포인트 수
    prediction_loss_only=True,
    logging_dir="./logs",           # 로그 저장 디렉토리
    logging_steps=100,              # 로그를 남길 단계 간격 (줄임)
)

# 7. Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# 8. 학습 시작
trainer.train()

# 9. 학습 완료된 모델 저장
trainer.save_model("./gpt2_finetuned")
tokenizer.save_pretrained("./gpt2_finetuned")

print("GPT-2 모델 미세 조정 완료 및 저장되었습니다!")


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
100,1.943800
200,1.695200
300,1.596200
400,1.536800
500,1.488700
600,1.454100
700,1.416900
800,1.393100
900,1.378000
1000,1.351700


GPT-2 모델 미세 조정 완료 및 저장되었습니다!


1. Google Drive에 저장:

In [8]:
from google.colab import drive
drive.mount('/content/drive')

# 저장 경로 설정 (Google Drive)
output_dir = '/content/drive/MyDrive/gpt2_finetuned2'

# 모델과 토크나이저를 Google Drive에 저장
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/gpt2_finetuned2/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2_finetuned2/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2_finetuned2/vocab.json',
 '/content/drive/MyDrive/gpt2_finetuned2/merges.txt',
 '/content/drive/MyDrive/gpt2_finetuned2/added_tokens.json')

2. 코랩을 재실행했을 때 미세 조정된 모델을 불러오는 방법:

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Google Drive에서 저장된 모델과 토크나이저 로드
model_path = '/content/drive/MyDrive/gpt2_finetuned'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# GPU로 모델 이동 (선택 사항)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("미세 조정된 GPT-2 모델을 로드했습니다.")


In [12]:
# !pip install streamlit

In [37]:
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 1. 미세 조정된 GPT-2 모델과 토크나이저 로드
model_path = './gpt2_finetuned'  # 미세 조정된 모델 경로

# 모델과 토크나이저를 로드하는 과정에서 오류가 발생할 수 있으므로 예외 처리
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    st.write("모델이 성공적으로 로드되었습니다.")

    # GPU가 사용 가능한 경우 GPU로 모델 이동
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

except Exception as e:
    st.write(f"모델 로드 중 오류 발생: {str(e)}")
    model = None  # 모델 로드 실패 시 model 변수를 None으로 설정

# 2. 질문에 대한 답변 생성 함수 정의
def generate_answer(question, model, tokenizer, max_length=100, num_return_sequences=1):
    """
    질문을 입력받아 GPT-2 모델이 답변을 생성하도록 하는 함수
    """
    if model is None:
        return "모델이 로드되지 않았습니다."

    # 질문을 토큰화
    inputs = tokenizer.encode(question, return_tensors="pt").to(device)

    # GPT-2 모델을 사용하여 답변 생성
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # 토큰을 다시 텍스트로 변환
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# 3. Streamlit 인터페이스 설정
st.title("GPT-2 질문-답변 시스템")

# 질문 입력
question = st.text_input("질문을 입력하세요")

# 답변을 생성하는 버튼
if st.button("답변 생성"):
    if question:
        answer = generate_answer(question, model, tokenizer)
        st.write(f"GPT-2 답변: {answer}")
    else:
        st.write("질문을 입력해 주세요.")


2024-10-14 06:14:53.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.953 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 06:14:53.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [39]:
%%writefile app.py
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 1. 미세 조정된 GPT-2 모델과 토크나이저 로드
model_path = './gpt2_finetuned'  # 미세 조정된 모델 경로

# 모델과 토크나이저를 로드하는 과정에서 오류가 발생할 수 있으므로 예외 처리
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    st.write("모델이 성공적으로 로드되었습니다.")

    # GPU가 사용 가능한 경우 GPU로 모델 이동
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

except Exception as e:
    st.write(f"모델 로드 중 오류 발생: {str(e)}")
    model = None  # 모델 로드 실패 시 model 변수를 None으로 설정

# 2. 질문에 대한 답변 생성 함수 정의
def generate_answer(question, model, tokenizer, max_length=100, num_return_sequences=1):
    """
    질문을 입력받아 GPT-2 모델이 답변을 생성하도록 하는 함수
    """
    if model is None:
        return "모델이 로드되지 않았습니다."

    # 질문을 토큰화
    inputs = tokenizer.encode(question, return_tensors="pt").to(device)

    # GPT-2 모델을 사용하여 답변 생성
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # 토큰을 다시 텍스트로 변환
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# 3. Streamlit 인터페이스 설정
st.title("GPT-2 질문-답변 시스템")

# 질문 입력
question = st.text_input("질문을 입력하세요")

# 답변을 생성하는 버튼
if st.button("답변 생성"):
    if question:
        answer = generate_answer(question, model, tokenizer)
        st.write(f"GPT-2 답변: {answer}")
    else:
        st.write("질문을 입력해 주세요.")


Overwriting app.py


In [26]:
!ngrok authtoken 2nPpbV01vqVUJlachQwY0nyTTWn_k9RSDqfvJukpN4L3xvoB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [30]:
!curl ifconfig.me

34.46.68.91

In [ ]:
!streamlit run app.py & yes | npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.46.68.91:8501

your url is: https://cold-wasps-happen.loca.lt
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Google Drive에서 저장된 모델과 토크나이저 로드
output_dir = '/content/drive/MyDrive/gpt2_finetuned'
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

# GPU가 사용 가능한 경우 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 모델을 평가 모드로 전환
model.eval()

# 질문-답변을 위한 함수 정의
def generate_answer(question, max_length=150):
    # 입력 텍스트를 토큰화
    inputs = tokenizer(question, return_tensors="pt").to(device)

    # 모델을 통해 답변 생성
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,  # 생성할 텍스트의 최대 길이
            num_return_sequences=1,  # 한 번에 몇 개의 답변을 생성할지
            repetition_penalty=1.5,  # 반복 억제
            no_repeat_ngram_size=3,  # 같은 n-그램 반복을 피하도록 설정
            pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 설정
            top_p=0.85,  # 상위 p%의 확률을 가진 토큰만 고려
            temperature=0.6,  # 텍스트 생성의 무작위성을 제어하는 매개변수 (낮추면 보수적)
        )

    # 토큰을 텍스트로 변환하여 출력
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return answer

# 테스트 질문
question = "정부의 새로운 정책은 무엇인가요?"

# 답변 생성
generated_answer = generate_answer(question)
print(f"GPT-2 답변: {generated_answer}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


GPT-2 답변: 정부의 새로운 정책은 무엇인가요?
답병: 그런데 일단 지경하는, 자칫 사실은 얻을 테니꿔…… (「직원도」) 1ㆍ2’ ‘1㆓3% 이상. 2ﾎ4%, 3�5%. 4�


In [ ]:
# !pip install streamlit

In [ ]:
!cp -r /content/drive/MyDrive/gpt2_finetuned /content/gpt2_finetuned

In [ ]:
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 1. 미세 조정된 GPT-2 모델과 토크나이저 로드
model_path = '/content/gpt2_finetuned'  # 미세 조정된 모델 경로

# 모델과 토크나이저를 로드하는 과정에서 오류가 발생할 수 있으므로 예외 처리
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    st.write("모델이 성공적으로 로드되었습니다.")

    # GPU가 사용 가능한 경우 GPU로 모델 이동
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

except Exception as e:
    st.write(f"모델 로드 중 오류 발생: {str(e)}")
    model = None  # 모델 로드 실패 시 model 변수를 None으로 설정

# 2. 질문에 대한 답변 생성 함수 정의
def generate_answer(question, model, tokenizer, max_length=100, num_return_sequences=1):
    """
    질문을 입력받아 GPT-2 모델이 답변을 생성하도록 하는 함수
    """
    if model is None:
        return "모델이 로드되지 않았습니다."

    # 질문을 토큰화
    inputs = tokenizer.encode(question, return_tensors="pt").to(device)

    # GPT-2 모델을 사용하여 답변 생성
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # 토큰을 다시 텍스트로 변환
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# 3. Streamlit 인터페이스 설정
st.title("GPT-2 질문-답변 시스템")

# 질문 입력
question = st.text_input("질문을 입력하세요")

if st.button("답변 생성"):
    # GPT-2 모델을 사용하여 답변 생성
    if question:
        answer = generate_answer(question, model, tokenizer)
        st.write(f"GPT-2 답변: {answer}")
    else:
        st.write("질문을 입력해주세요.")


2024-10-13 17:02:11.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.415 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.607 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-13 17:02:11.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

우분투, 터미널에서 실행

1. nano streamlit_chatbot4.py
2. streamlit run streamlit_chatbot4.py



1. 코드를 파일로 저장하기

In [ ]:
code = '''
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 1. 미세 조정된 GPT-2 모델과 토크나이저 로드
model_path = "./gpt2_finetuned"  # 미세 조정된 모델 경로
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# GPU가 사용 가능한 경우 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. 질문에 대한 답변 생성 함수 정의
def generate_answer(question, model, tokenizer, max_length=100, num_return_sequences=1):
    inputs = tokenizer.encode(question, return_tensors="pt").to(device)

    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# 3. Streamlit 인터페이스 설정
st.title("GPT-2 질문-답변 시스템")

# 질문 입력
question = st.text_input("질문을 입력하세요")

if st.button("답변 생성"):
    if question:
        answer = generate_answer(question, model, tokenizer)
        st.write(f"GPT-2 답변: {answer}")
    else:
        st.write("질문을 입력해주세요.")
'''

# streamlit_chatbot.py 파일로 저장
with open('streamlit_chatbot.py', 'w') as f:
    f.write(code)

print("streamlit_chatbot.py 파일이 저장되었습니다.")


In [ ]:
!ls


In [ ]:
!streamlit run streamlit_chatbot.py
